# Objetivos deste trabalho
- Familiarizar-se com a biblioteca PyTorch
- Definir arquiteturas MLP simples em PyTorch
- Treinar utilizando CIFAR10, testando diferentes arquiteturas, parâmetros, funções de loss e otimizadores
- Comparar os resultados obtidos utilizando apenas Perpceptrons

In [0]:
%matplotlib inline

import numpy as np 
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torch.utils.data import DataLoader

import torchvision
import torchvision.transforms as transforms
from torch.utils.data import random_split

from random import shuffle
import time
import copy

In [0]:
# Carregar os datasets

transform=transforms.Compose([
    #transforms.Grayscale(num_output_channels=1),

    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

dataset_train = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)

dataset_test = torchvision.datasets.CIFAR10(root='./data', train=False,
                                        download=True, transform=transform)

test, val = random_split(dataset_test, lengths = (5000,5000))


Files already downloaded and verified
Files already downloaded and verified


In [0]:
device =  torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [0]:
train_loader = DataLoader(dataset=dataset_train, shuffle=True)
test_loader = DataLoader(dataset=test, shuffle=False)
val_loader = DataLoader(dataset=val, shuffle=False)
dataloaders = {'train': train_loader, 'val':val_loader, 'test' : test_loader }
dataset_sizes = {'train' : len(train_loader.dataset), 'test' : len(test_loader.dataset), 'val': len(val_loader.dataset)}

In [0]:
# Definir a arquitetura MLP

class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(32*32, 20)
        self.fc2 = nn.Linear(20, 10)
        self.activation_function = nn.Sigmoid()
    def forward(self, x):
        x = x.view(-1, 32*32)
        x = self.activation_function(self.fc1(x))
        x = self.activation_function(self.fc2(x))
        return x

In [0]:
# Realizar o treinamento aqui
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
  since = time.time()

  best_model_wts = copy.deepcopy(model.state_dict())
  best_acc = 0.0

  for epoch in range(num_epochs):
      print('Epoch {}/{}'.format(epoch, num_epochs - 1))
      print('-' * 10)

      # Each epoch has a training and validation phase
      for phase in ['train', 'val']:
          if phase == 'train':
              model.train()  # Set model to training mode
          else:
              model.eval()   # Set model to evaluate mode

          running_loss = 0.0
          running_corrects = 0

          # Iterate over data.
          for inputs, labels in dataloaders[phase]:
              inputs = inputs.to(device)
              labels = labels.to(device)

              # zero the parameter gradients
              optimizer.zero_grad()

              # forward
              # track history if only in train
              with torch.set_grad_enabled(phase == 'train'):
                  outputs = model(inputs)
                  _, preds = torch.max(outputs, 1)
                  loss = criterion(outputs, labels)

                  # backward + optimize only if in training phase
                  if phase == 'train':
                      loss.backward()
                      optimizer.step()

              # statistics
              running_loss += loss.item() * inputs.size(0)
              running_corrects += torch.sum(preds == labels.data)
          if phase == 'train':
              scheduler.step()

          epoch_loss = running_loss / dataset_sizes[phase]
          epoch_acc = running_corrects.double() / dataset_sizes[phase]

          print('{} Loss: {:.4f} Acc: {:.4f}'.format(
              phase, epoch_loss, epoch_acc))

          # deep copy the model
          if phase == 'val' and epoch_acc > best_acc:
              best_acc = epoch_acc
              best_model_wts = copy.deepcopy(model.state_dict())

      print()

  time_elapsed = time.time() - since
  print('Training complete in {:.0f}m {:.0f}s'.format(
      time_elapsed // 60, time_elapsed % 60))
  print('Best val Acc: {:4f}'.format(best_acc))

  # load best model weights
  model.load_state_dict(best_model_wts)
  return model

In [0]:
# Avaliar o modelo aqui (no conjunto de teste)
def evaluate(model, data_loader):
  acc = 0.0
  model.eval()
  with torch.no_grad():
    for i, (inputs, labels) in enumerate(data_loader):
        inputs = inputs.to(device)
        labels = labels.to(device)

        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        acc += torch.sum(pred == label.data)

  print('Acc test_set: {:.4f}'.format(acc.double()/len(dataset_test)))

In [0]:
model = MLP()
print(model)

MLP(
  (fc1): Linear(in_features=1024, out_features=20, bias=True)
  (fc2): Linear(in_features=20, out_features=10, bias=True)
  (activation_function): Sigmoid()
)


In [0]:
# Definir otimizador e loss
# Nota: testar outros otimizadores e funções de loss (em particular cross entropy)

optimizer = torch.optim.SGD(model.parameters(), lr = 0.001)
criterion = torch.nn.CrossEntropyLoss()
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size = 4)
num_epochs = 10

model.to(device)

MLP(
  (fc1): Linear(in_features=1024, out_features=20, bias=True)
  (fc2): Linear(in_features=20, out_features=10, bias=True)
  (activation_function): Sigmoid()
)

In [0]:
train_model(model, criterion, optimizer, scheduler, num_epochs )

Epoch 0/9
----------
train Loss: 2.2966 Acc: 0.1374
val Loss: 2.2878 Acc: 0.1824

Epoch 1/9
----------
train Loss: 2.2740 Acc: 0.1915
val Loss: 2.2607 Acc: 0.1890

Epoch 2/9
----------
train Loss: 2.2465 Acc: 0.1956
val Loss: 2.2370 Acc: 0.1860

Epoch 3/9
----------
train Loss: 2.2264 Acc: 0.1991
val Loss: 2.2217 Acc: 0.2062

Epoch 4/9
----------
train Loss: 2.2179 Acc: 0.2082
val Loss: 2.2198 Acc: 0.2092

Epoch 5/9
----------
train Loss: 2.2165 Acc: 0.2076
val Loss: 2.2186 Acc: 0.2086

Epoch 6/9
----------
train Loss: 2.2152 Acc: 0.2091
val Loss: 2.2174 Acc: 0.2112

Epoch 7/9
----------
train Loss: 2.2140 Acc: 0.2097
val Loss: 2.2162 Acc: 0.2118

Epoch 8/9
----------
train Loss: 2.2133 Acc: 0.2108
val Loss: 2.2161 Acc: 0.2114

Epoch 9/9
----------
train Loss: 2.2132 Acc: 0.2109
val Loss: 2.2160 Acc: 0.2112

Training complete in 9m 40s
Best val Acc: 0.211800


MLP(
  (fc1): Linear(in_features=1024, out_features=20, bias=True)
  (fc2): Linear(in_features=20, out_features=10, bias=True)
  (activation_function): Sigmoid()
)